In [1]:
#Needed PIP installs: rdkit, pandas, scipy, matplotlib, openpyxl (needed for Tyler)

#Further program ideas
#1 Function to limit dendrogram to a specific number, rather than % similarity
#2 Function to sort the similarities (high to low), then output the table as an Excel file
#3 Function to save the df of compounds with fingerprints (before tanimoto similarity step), then reuse the computed df if it has already been computed before.

import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
import pandas as pd
#############################################
############################################
############################################

import tkinter as tk
import os
from tkinter import *
from tkinter import filedialog as fd
from tkinter.messagebox import showinfo

import io
from io import StringIO

import ntpath
import time
import sys, math


## _____INPUTS_window__________Take in all of the parameters

root= tk.Tk()
root.title("      Structure Similarity Search")
btn21 = StringVar()
btn23 = StringVar()
btn30 = StringVar()
ent1 = tk.Entry(root,font='serif 10')
ent1.grid(row=2,column=1,ipadx=70,sticky=W+E)

ent2 = tk.Entry(root,font='serif 10')
ent2.grid(row=4,column=1,ipadx=70,sticky=W+E)

root.lift()
root.attributes('-topmost',True)
root.after_idle(root.attributes,'-topmost',False)

def select_MiBiG():
    global MiBiG_directory
    global MiBiG_filepath
    global MiBiG_filename
        
    filetypes1 = (("CSV files", '*.csv'),("All files", '*.*'))

    MiBiG_filepath = fd.askopenfilename(
        title="Open MiBiG file",
        #initialdir='/Users/tngraf/Downloads/Structure_similarity_search_tutorial/Structure_similarity_search_tutorial/',
        initialdir='/Users/Karin/Documents/A_fischeri/Figshare/Structure_similarity_search_tutorial/Structure_similarity_software_edited',
        filetypes=filetypes1)
    MiBiG_filename = ntpath.basename(MiBiG_filepath)
    ent1.insert(tk.END, MiBiG_filename) # add this
    print(MiBiG_filepath,"\n")
    MiBiG_directory = ntpath.dirname(MiBiG_filepath[0])
    file_name = os.path.basename(MiBiG_filepath[0])
        
    
def get_SMILES():
    global SMILES_data
    global import_cmpd_file_path
    
    filetypes2 = (("TXT files", '*.txt'),("All files", '*.*'))
    import_cmpd_file_path = fd.askopenfilename(
        title="Open file(s)",
        #initialdir='/Users/tngraf/Downloads/Structure_similarity_search_tutorial/Structure_similarity_search_tutorial/',
        initialdir='/Users/Karin/Documents/A_fischeri/Figshare/Structure_similarity_search_tutorial/',
        filetypes=filetypes2)
    Cmpd_filename = ntpath.basename(import_cmpd_file_path)
    ent2.insert(tk.END, Cmpd_filename) # add this
    SMILES_data = pd.read_csv(import_cmpd_file_path, header=None, names=['SMILES', 'Name'])

def Button_fun():

    global Dendro_limit
    global Dendro_decimal
    global Dendro_select
    
    Dendro_limit = entry22.get()
    Dendro_select = btn21.get()

    def isfloat(value):
        try:
            float(value)
            return True
        except ValueError:
            return False

    def is_int(value):
        try:
            int(value)
            return True
        except ValueError:
            return False

    if Dendro_limit == "": #for when there is no input, it designates a default value
        Dendro_limit = 50
    elif ((type(Dendro_limit) == str) & (isfloat(Dendro_limit) == True)):
        Dendro_decimal = float(Dendro_limit)
    elif ((type(Dendro_limit) == str) & (is_int(Dendro_limit) == True)):
        Dendro_limit = int(Dendro_limit)
    else:
        print(Dendro_limit, " is an invalid entry.  Try again.")
        
    Dendro_decimal = float(Dendro_limit) / 100  ## converts the ppm value input above into decimal form
    
    root.destroy()


###### All operations should be above this line
#### Start of the GUI ####
Gap01 = Label(root)
Gap01.grid(row=1, columnspan=3)

cmpd_button=tk.Button(root,text="     MiBiG file (*.csv)     ", bg='green', fg='white', command=select_MiBiG)
#cmpd_button.grid(row=2, columnspan=3)
cmpd_button.grid(row=2, sticky=E, padx=7)
Gap03= Label(root)
Gap03.grid(row=3, columnspan=3)

cmpd_button=tk.Button(root,text="     SMILES file (*.txt)     ", bg='green', fg='white', command=get_SMILES)
cmpd_button.grid(row=4, sticky=E, padx=7)
Gap05 = Label(root)
Gap05.grid(row=5, columnspan=3)

Gap20 = Label(root)
Gap20.grid(row=20, columnspan=3)
labelS2 = Label(root, text="_______Special processing settings______________________")
labelS2.grid(row=21)

label21 = Label(root, text="Limit dendrogram:")
label21.grid(row=22, column=0, sticky=E)
btn21.set("no")
button21=Radiobutton(root,text="Yes",value = "yes",variable = btn21).grid(row=23, column=0, sticky=E)
button21=Radiobutton(root,text="No",value = "no",variable = btn21).grid(row=23, column=1, sticky=W)

label22 = Label(root, text="Similarity limit for dendrogram (default = 50%): ")
label22.grid(row=24, column=0, sticky=E)
entry22 = Entry(root)
entry22.grid(row=24, column=1, pady=2, sticky=W)

Gap37 = Label(root)
Gap37.grid(row=37, columnspan=3)

CMPD_LIST = Button(root, text="     Start processing     ", fg='black', bg='lightsteelblue2', relief='raised', command=Button_fun)
CMPD_LIST.grid(row=38, columnspan=3)

Gap39 = Label(root)
Gap39.grid(row=39, columnspan=3)

root.mainloop()

#### End of the GUI ####
#### ----------------------------- Above are all the data inputs --------------

start_time = time.time()  ## Start of time function to measure how long processing takes
    
data_MiBiG = pd.read_csv(MiBiG_filepath, sep='\t', header=None, names=['SMILES', 'Name'])

print("Input accepted for MiBiG file:\n  %s \n" % (MiBiG_filename))
#print(data_MiBiG.head(10),"\n\n\n")

print("data_MiBiG shape",data_MiBiG.shape)
print("\n")

#########################################
#########################################
##########################################

print("Dendro_limit: ",Dendro_limit)
print("Dendro_decimal: ",Dendro_decimal)
print("Dendro_select: ",Dendro_select,"\n")


#2 Check SMILES
# Assuming data_MiBiG is your DataFrame with 'SMILES' column
data_MiBiG['Molecule'] = data_MiBiG['SMILES'].apply(Chem.MolFromSmiles)

# Check for invalid SMILES
invalid_smiles = data_MiBiG[data_MiBiG['Molecule'].isnull()]

# Display invalid SMILES, if any
if(len(invalid_smiles)>0):
   print("Invalid SMILES:")
   print(invalid_smiles[['SMILES', 'Name']])


#5 Let's add our experimental structure to one of the data frames
SMILES_len = len(SMILES_data)
print("Smiles data:",SMILES_data.head(),"\n",SMILES_len,"\n\n")

# Append using concat (preferred over data_MiBiG.append, which is deprecated)
data_MiBiG = pd.concat([data_MiBiG, SMILES_data], ignore_index=True)
data_MiBiG['Molecule'] = data_MiBiG['SMILES'].apply(Chem.MolFromSmiles) #


from rdkit.Chem import AllChem
#from rdkit.Chem import rdFingerprintGenerator
#data_MiBiG['Fingerprint'] = data_MiBiG['SMILES'].apply(lambda x: rdFingerprintGenerator.GetMorganGenerator(Chem.MolFromSmiles(x), 2, nBits=2048))
#fp = mfpgen.GetFingerprint(ms[0])

# Add a new column for fingerprints
data_MiBiG['Fingerprint'] = data_MiBiG['SMILES'].apply(lambda x: AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(x), 2, nBits=2048))

# Convert fingerprints to binary strings
data_MiBiG['Fingerprint'] = data_MiBiG['Fingerprint'].apply(lambda x: x.ToBitString())

#7 What does the 'finderprint' of the newly added compound look like? 
#MiBiG_length = len(data_MiBiG) -1
#fp = data_MiBiG.loc[MiBiG_length, "Fingerprint"] # fingerprint for the last entry in the dataframe data_MiBiG
#print("\nfingerprint\n")
#print(fp)


Mid1_time = time.time()  ## Start of dendrogram function to measure how long dendrogram takes
print("---Time to complete fingerprinting: %s seconds ---" % (Mid1_time - start_time))  ## Indicates amount of time it took after inputs were given
#MiBiG_save_name = [name.split(".csv")[0] for name in MiBiG_filename]
#MiBiG_save_name += ".fngpt" # Fingerprints file name for output
data_MiBiG.to_csv("MiBiG_data_Fingerprint.fgnpt", sep='\t', header=True, index = False)  ## WRITE Results for if there was only 1 input file
                
#8 2. Calculate Tanimoto Similarity Matrix
from rdkit import DataStructs
# Create a function to calculate Tanimoto similarity
def calculate_tanimoto(fp1, fp2):
    return DataStructs.FingerprintSimilarity(fp1, fp2)

# Create a similarity matrix
similarity_matrix = []

for i in range(len(data_MiBiG)):
    row_similarities = []
    for j in range(len(data_MiBiG)):
        fp1 = DataStructs.CreateFromBitString(data_MiBiG.iloc[i]['Fingerprint'])
        fp2 = DataStructs.CreateFromBitString(data_MiBiG.iloc[j]['Fingerprint'])
        similarity = calculate_tanimoto(fp1, fp2)
        row_similarities.append(similarity)
    similarity_matrix.append(row_similarities)

print(type(similarity_matrix),"\nSIM\n")
#print(similarity_matrix)

#10 What does this data matrix look like?
similarity_df=pd.DataFrame(similarity_matrix)
similarity_length = len(similarity_df) -1
similarity_df.iloc[:, 0] # first column
similarity_df.iloc[:, similarity_length] # last column

print("similarity_df shape before",similarity_df.shape)
#print(similarity_df.head(10),"\n\nX11X\n\n\n")


#11Let's set row and column names for convenience
similarity_df.columns = data_MiBiG["Name"]  # Set columns names
similarity_df.index = data_MiBiG["Name"]    # Set row names

print("similarity_df shape after column renamed",similarity_df.shape)
#print(similarity_df.head(10),"\n\nX1b1bX\n\n\n")

# Which compounds is "our compound" most similar to?
row_values = similarity_df.iloc[similarity_length]
top_5 = row_values.nlargest(5)
print("\n\n",top_5,"\n\n")

if (Dendro_select == "yes"): #### Sorts data by peak area to put biggest hits at top
    similarity_df = similarity_df.rename(columns={similarity_df.columns[similarity_length]: "SM_column"})

    similarity_df.drop(similarity_df.loc[similarity_df["SM_column"]<=Dendro_decimal].index, inplace=True)  ##THIS IS THE FUNCTION TO REMOVE VALUES BELOW "Threshold" FROM ms_data
    similarity_matrix = similarity_df.values.tolist()
    similarity_df['Name'] = similarity_df.index
    print(similarity_df.head(5),"\n\nX22X\n")
    print("similarity_df--shape after limitation by Dendro_decimal",similarity_df.shape)
    print("\nX33X\n\n")
#    print(similarity_matrix,"\n\nX44X\n\n\n")

Mid2_time = time.time()  ## Start of dendrogram function to measure how long dendrogram takes
print("---Time to complete similarity analysis: %s seconds ---" % (Mid2_time - Mid1_time))  ## Indicates amount of time it took after inputs were given

#15 3. Hierarchical Clustering
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt

# Perform hierarchical clustering
cluster_data = linkage(similarity_matrix, method='average', metric='euclidean')

print("similarity_df shape ",similarity_df.shape)
print("\nYYYY\n")
print(cluster_data,"\nCLUSTER\n\n")

# Plot dendrogram
similarity_length = len(similarity_df) -1

wide = 10
tall = int(similarity_length/4) +1
plt.figure(figsize=(wide, tall))
dendrogram(cluster_data, labels=similarity_df['Name'].tolist(), orientation='right', leaf_font_size=10)
plt.title('Molecule Clustering based on Tanimoto Similarity')
#plt.savefig('/Users/tngraf/Downloads/Structure_similarity_search_tutorial/Structure_similarity_search_tutorial/mibig4_testing_clustering.pdf', format='pdf', bbox_inches='tight')
plt.savefig('/Users/karin/Documents/A_fischeri/Figshare/Structure_similarity_search_tutorial/Structure_similarity_software_edited/mibig4_testing_clustering.pdf', format='pdf', bbox_inches='tight')

#plt.show()

print("---Time to make fingerprints: %s seconds ---" % (Mid1_time - start_time))  ## Indicates amount of time it took after inputs were given
print("---Time to make similarities: %s seconds ---" % (Mid2_time - Mid1_time))  ## Indicates amount of time it took after inputs were given
print("---Time to make dendrogram: %s seconds ---" % (time.time() - Mid2_time))  ## Indicates amount of time it took after inputs were given
print("---Total processing took: %s seconds ---" % (time.time() - start_time))  ## Indicates amount of time it took after inputs were given

#For larger sets of SMILES, instead of manually appending them with python,
#I just generated a new data frame combining the database I wanted to use and then pasting my SMILES and Names.

## https://github.com/rdkit/rdkit/issues/7625 is about deprecation issue with AllChem.GetMorganFingerprintAsBitVect

2025-05-06 14:18:02.659 python[44037:14289338] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'


/Users/karin/Documents/A_fischeri/Figshare/code/Tyler/Re_ Structure similarity search script/MIBiGv4.0_SMILES_BGC_acc-names-FUNGAL-only.csv 

Input accepted for MiBiG file:
  MIBiGv4.0_SMILES_BGC_acc-names-FUNGAL-only.csv 

data_MiBiG shape (1092, 2)


Dendro_limit:  50
Dendro_decimal:  0.5
Dendro_select:  yes 

Smiles data:                                               SMILES  Name
0  CCC[C@@H]1[C@@]2(C(OC)=O)C(C(O)=O)=CC[C@@H](C(...   NaN 
 1 


---Time to complete fingerprinting: 0.6915581226348877 seconds ---
<class 'list'> 
SIM

similarity_df shape before (1093, 1093)
similarity_df shape after column renamed (1093, 1093)


 Name
NaN                                        1.000000
BGC0003041_MIBiGv4.0_sativene              0.240964
BGC0003041_MIBiGv4.0_prehelminthosporol    0.235955
BGC0002651_MIBiGv4.0_sordaricin B          0.212121
BGC0002956_MIBiGv4.0_betaestacin Vc        0.205607
dtype: float64 


Name  BGC0000003_MIBiGv4.0_AF-toxin  BGC0000004_MIBiGv4.0_aflatoxin G1  \
Name  

ValueError: The number of observations cannot be determined on an empty distance matrix.